In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
features=pd.read_csv('training_set_features.csv')
labels=pd.read_csv('training_set_labels.csv')

In [3]:
labels1=labels.drop('xyz_vaccine',axis=1)

In [4]:
data=features.merge(labels1,how='outer')

In [5]:
data.drop(['health_insurance','employment_industry','employment_occupation','respondent_id','hhs_geo_region'],axis=1,inplace=True)

In [6]:
X=data.drop('seasonal_vaccine',axis=1)
y=data['seasonal_vaccine']

In [7]:
best_labels=['xyz_concern',
 'xyz_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_xyz',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'opinion_xyz_vacc_effective',
 'opinion_xyz_risk',
 'opinion_xyz_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'census_msa',
 'household_adults',
 'household_children']

In [10]:
X_new=X[best_labels]

In [11]:
num_cols_new=[i for i in X_new if X_new[i].dtype!=object]
cat_cols_new=[i for i in X_new if X_new[i].dtype==object]

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [14]:
num_pipe=make_pipeline(SimpleImputer(strategy='most_frequent'))
cat_pipe=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())

In [16]:
col_trans=make_column_transformer((num_pipe,num_cols_new),(cat_pipe,cat_cols_new),remainder='passthrough')

In [17]:
from sklearn.svm import SVC

In [31]:
pipe=make_pipeline(col_trans,SVC(probability=True,C=1,kernel='rbf'))

In [28]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(pipe,X_new,y,cv=10,scoring='roc_auc').mean()

In [32]:
pipe.fit(X_new,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['xyz_concern',
                                                   'xyz_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavio...
                                                   'opinion_seas_sick_from_vacc',
                                                   'household_adults',
                                                   'household_children']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['age_group', 'education',
                                                   'race', 'sex',
                                                   'income_poverty',
                                                   'marital_status',
                                                   'rent_or_own',
                                                   'employment_status',
                                                   'census_msa'])])),
                ('svc', SVC(C=1, probability=True))])

In [33]:
test_data=pd.read_csv('test_set_features.csv')

In [34]:
test=test_data.drop(['health_insurance','employment_industry','employment_occupation','respondent_id','hhs_geo_region'],axis=1)

In [35]:
prob=pipe.predict_proba(test)[:,1]

In [36]:
seasonal=pd.DataFrame(prob,columns=['seasonal_vaccine'])

In [37]:
seasonal

,seasonal_vaccine
0,0.182662
1,0.081472
2,0.840734
3,0.885713
4,0.422715
...,...
26703,0.449064
26704,0.196040
26705,0.131785
26706,0.367012


In [38]:
seasonal.to_csv('seasonal_vaccine.csv')